In [1]:
from vn_backtest import *
import numpy as np
import pandas as pd
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from datetime import datetime,timedelta
import numpy as np
import time

In [2]:
# Database connection parameters
db_params = {
    "host": "api.algotrade.vn",
    "port": 5432,
    "database": "algotradeDB",
    "user": "intern_read_only",
    "password": "ZmDaLzFf8pg5"
}

connection = psycopg2.connect(**db_params)
def execute_query(query, from_date, to_date):
    cursor = connection.cursor()
    try:
        cursor.execute(query, (from_date, to_date))
        result = cursor.fetchall()

        cursor.close()
        return result

    except Exception as e:
        print(f"Error: {e}")

In [9]:
import pandas as pd
from datetime import datetime, time

def get_stock_info(symbol, startDate, endDate):
    query = """
        select m.datetime, m.tickersymbol, m.price
        from quote.matched m
        where m.tickersymbol = %s and
        m.datetime between %s and %s and
        ((m.datetime::TIME BETWEEN '09:00:00' AND '11:30:00')
        or (m.datetime::TIME BETWEEN '13:00:00' AND '14:45:00'))
    """
    cursor = connection.cursor()
    try:
        cursor.execute(query, (symbol, startDate, endDate))
        result = cursor.fetchall()
        cursor.close()
    except Exception as e:
        print(f"Error: {e}")
        return None
    columns = ["datetime", "tickersymbol", "price"]
    matched = pd.DataFrame(result, columns=columns)
    matched = matched.astype({"price": float})
    return matched

def get_closing_prices(stocks, startDate, endDate):
    # Initialize an empty DataFrame to store all results
    all_results = pd.DataFrame()
    
    for symbol in stocks:
        # Get stock data
        df = get_stock_info(symbol, startDate, endDate)
        
        if df is None or df.empty:
            continue
            
        # Convert datetime to datetime object
        df['datetime'] = pd.to_datetime(df['datetime'])
        # Extract date for grouping
        df['date'] = df['datetime'].dt.date
        # Create period identifier (morning = 1, afternoon = 2)
        df['period'] = df['datetime'].apply(
            lambda x: 1 if x.time() <= pd.Timestamp('11:30:00').time() 
            else 2
        )
        
        # Get the last price for each period of each day
        closing_prices = df.groupby(['date', 'period']).agg({
            'price': 'last',
            'tickersymbol': 'first'
        }).reset_index()
        
        # Convert dates to datetime with specific closing times
        closing_prices['datetime'] = closing_prices.apply(
            lambda row: pd.Timestamp.combine(
                row['date'], 
                time(11, 30) if row['period'] == 1 else time(14, 30)
            ),
            axis=1
        )
        
        # Add morning indicator (1 for morning, 0 for afternoon)
        closing_prices['morning'] = closing_prices['period'].apply(lambda x: 1 if x == 1 else 0)
        
        # Create a pivot table with datetime as index and stocks as columns
        symbol_prices = closing_prices.pivot(
            index='datetime',
            columns='tickersymbol',
            values='price'
        )
        
        # If this is the first stock, add the morning column
        if all_results.empty:
            symbol_prices = symbol_prices.reset_index()
            symbol_prices['morning'] = closing_prices['morning'].values
            symbol_prices = symbol_prices.set_index('datetime')
        
        # Merge with main DataFrame
        if all_results.empty:
            all_results = symbol_prices
        else:
            all_results = pd.concat([all_results, symbol_prices], axis=1)
    
    # Reset index to make datetime a column
    all_results = all_results.reset_index()
    return all_results

In [10]:
# Define your parameters
stocks = ['VN30', 'FUEVFVND', "FUESSVFL", "E1VFVN30", "FUEVN100", "FUEVN50"]
startDate = '2020-01-01'  # Adjust these dates as needed
endDate = '2025-02-27'    # Today is Feb 27, 2025 based on system date

# Get the data
result_df = get_closing_prices(stocks, startDate, endDate)



In [11]:
# Display the result
result_df.to_csv('stock_data.csv')